Project 2

Rishi Shah (rrs141 Section 05)
Sneh Shah (sjs413 Section 02)

In [ ]:
import numpy as np
import scipy as sps
import sys
from matplotlib import pyplot as plt
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Latex
from numpy import linalg
from scipy import linalg, stats
from sympy import *
import time
import pandas as pd
import random

In [ ]:
#d by d grid with n number of mines
# 9 = Bomb
#0-8 = number of surrounding bombs


def genGrid(d, n):  
  grid = np.zeros((d,d)) 
  count = 0

  while count<n:
    x =  np.random.randint(0,d) #generating random coordinate for bomb 
  
    y = np.random.randint(0,d)
    

    if grid[x][y] == 9: #if coordinate is already bomb, don't increase counter and find new coordinate 
      continue 

    grid[x][y] = 9 #else set coordinate to bomb and increase count
    count+=1


  grid = countBombs(grid) #setting board clues 

  return grid

movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1) ]


def countBombs(grid):
  for row in range(len(grid)):  #going through whole grid and counting number of surrounding bombs to find clues 
    for col in range(len(grid[0])):
     
 
      if (grid[row][col]==9): #ignore if cell is already mine 
        continue

      for i in range(len(movements)): #iterating through all possible movements 
        distX, distY = movements[i]
        x = row + distX
        y = col + distY 

        if (x<0 or y < 0 or x >= len(grid) or y>=len(grid)): #if out of bounds, continue 
          continue
        
        if (grid[x][y]==9): #if neighboring cell is mine increment clue by 1 
          grid[row][col]+=1
    
  return grid
        

print(genGrid(3,1))

[[0. 0. 0.]
 [1. 1. 0.]
 [9. 1. 0.]]


# Basic Agent

In [ ]:
class cell:
  def __init__(self, status, clue, idSafe, idMine, hidden, total): 
    self.status = status #status of mine: 'C' = covered, 'M' = Mine, 'S' = Safe, 'F' = Flagged Safe/in fringe
    self.clue = clue #integer representing number of surrounding bombs 
    self.idSafe = idSafe #number of known safe neighbors
    self.idMine = idMine #number of known mine neighbors 
    self.hidden = hidden #number of hidden cells left 
    self.total = total #total number of neighbors 

In [ ]:
#this function determine how many neighbors are hidden, safe, or mines for given coordinate
def findNeighbors(game, map, row, col):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1) ]
  hiddenNeighbors = 0
  safe = 0
  mine = 0 

  for i in range(len(movements)):
    distX, distY = movements[i]
    x = row + distX
    y = col + distY 
    
  
    if (x<0 or y < 0 or x >= len(game) or y>=len(game)):
      continue

  
    if (game[x][y]==-1):
      hiddenNeighbors += 1
    
    if ((game[x][y]>=0 and game[x][y]<=8) or map[x][y].status == 'S'):
      safe+=1
    
    if (game[x][y] == 9 or map[x][y].status == 'M'):
      mine+=1

  return hiddenNeighbors, safe, mine

In [ ]:
def updateNeighbors(game, map, row, col, unexplored, explored, fringe):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1) ]
  current = game[row][col]
  d = len(game)
  #go through neighbors and update idMine and hidden
  if (current == 9 or map[row][col].status == 'M'):
    for i in range(len(movements)):
      distX, distY = movements[i]
      x = row + distX
      y = col + distY 
      
    
      if (x<0 or y < 0 or x >= len(game) or y>=len(game)):
        continue

      map[x][y].idMine += 1
      map[x][y].hidden -= 1

  elif (current == 0 or map[row][col].status == 'S'):

    #go through neighbors and update idSafe and hidden
    for i in range(len(movements)):
      distX, distY = movements[i]
      x = row + distX
      y = col + distY 
        
      
      if (x<0 or y < 0 or x >= len(game) or y>=len(game)):
        continue

      map[x][y].idSafe += 1
      map[x][y].hidden -= 1 

   
  return map, unexplored, explored, fringe, game

In [ ]:
#this function flags a cell as safe or mine
def flag(map, row, col, d, num, unexplored, explored, fringe, game, bombs):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1) ]
  for i in range(len(movements)):
      distX, distY = movements[i]
      x = row + distX
      y = col + distY 
      
      if (x<0 or y < 0 or x >= d or y >= d):
        continue

 
      #if hidden, flag accordingly
      if map[x][y].status == 'C':
        
        #flag as safe
        if num == 1:
          map[x][y].status = 'F'
          if (x,y) not in fringe and (x,y) not in explored: 
            fringe.append((x,y))

        #flag as mine
        else:
          map[x][y].status = 'M'
          unexplored.remove((x,y))
          explored.add((x,y))
          game[x][y] = 10
          bombs.append((x,y))
          map, unexplored, explored, fringe, game = updateNeighbors(game, map, x, y, unexplored, explored, fringe)

  return map, unexplored, explored, fringe, game, bombs

In [ ]:
#this function scans the board and determines if anything can be determined based on current clues
def scanBoard(map, unexplored, explored, fringe, game, bombs):
  for row in range(len(map)):
    for col in range(len(map[0])):
      current = map[row][col]

      #if current cell doesn't have hidden neighbors, continue since the status is known if it is not hidden
      if current.hidden == 0 or current.status == 'C' or current.status == 'F' or current.status == 'M':
        continue
      else:
        if map[row][col].clue - map[row][col].idMine == map[row][col].hidden:
          #flag remaining hidden neighbors as bombs
          map, unexplored, explored, fringe, game, bombs = flag(map, row, col, len(map), 0, unexplored, explored, fringe, game, bombs)
          return map, unexplored, explored, fringe, game, bombs

        elif (map[row][col].total - map[row][col].clue) - map[row][col].idSafe == map[row][col].hidden:
          #flag remaining neighbors as safe
          map, unexplored, explored, fringe, game, bombs = flag(map, row, col, len(map), 1, unexplored, explored, fringe, game, bombs)
  
  return map, unexplored, explored, fringe, game, bombs

In [ ]:
#this function generates a list of hidden neighbors for a given coordinate pair
def generateNeighbors(row, col, grid, fringe, explored):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1) ]
  neighbors = []
  for i in range(len(movements)):
    distX, distY = movements[i]
    x = row + distX
    y = col + distY 

    if (x<0 or y < 0 or x >= len(grid) or y>=len(grid) or (x,y) in fringe or (x,y) in explored):
      continue

    fringe.append((x,y))
  
  return fringe


In [ ]:
#this function is used to find a coordinate
def findCoordinate(map, unexplored):
  count = 0
  row = -1
  col = -1
  minimum = sys.maxsize
  for i in range(len(unexplored)):
    coordinate = unexplored[i]
    x = coordinate[0]
    y = coordinate[1]
    if map[x][y].hidden == map[x][y].total:
      row = x
      col = y
      break
    #sort unexplored coordinates by amount of neighboring mines that still need to be discovered
    else:
      total = 0;
      movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1)]
      for j in range(len(movements)):
        distX, distY = movements[j]
        xx = x + distX
        yy = y + distY 
        if (xx<0 or yy < 0 or xx >= len(map) or yy >= len(map)):
          continue
        #if the neighbor of the coordinate is safe, check how many mines that neighbor still has left to find
        if map[xx][yy].status == 'S':
          total += map[xx][yy].clue - map[xx][yy].idMine
      #update the row and col for coordinates with neighbors that have found most of their mines
      if minimum > total:
        minimum = total
        row = x
        col = y

  return row, col

In [ ]:
def basicAgent(grid):
  unexplored = []
  for row in range(len(grid)): 
    for col in range(len(grid[0])):
      unexplored.append((row,col))

  explored = set([])
  bombs = []
  explodedBombs = 0


  fringe = []
  d = len(grid)
  game = np.zeros((d,d)) 
  game[:] = -1

  #make a map of cells to represent the knowledge base
  map = np.zeros((d,d), dtype='object')
  for row in range(d):
    for col in range(d):

      if ((row, col) == (0, 0)) or  ((row, col) == (0, d-1)) or ((row, col) == (d-1, 0)) or ((row, col) == (d-1, d-1)):
        map[row][col] = cell('C',0,0,0, 3, 3)
      elif (row == 0 or row == d-1 or col == 0 or col == d-1):
        map[row][col] = cell('C',0,0,0, 5, 5) 
      else:
        map[row][col] = cell('C',0,0,0, 8, 8)
  
  
  while len(explored) != d*d:

    #there are no known safe cells
    if (len(fringe)==0):
      
      num = random.randint(0, len(unexplored)-1)
      coord = unexplored[num]
      x = coord[0]
      y = coord[1]

    #there are still safe cells left to explore
    else:
     
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

    if (x,y) in explored:
      continue

 
   

    if map[x][y].status == 'M':
      continue
    
    if grid[x][y] == 9:
      explodedBombs += 1
      game[x][y] = grid[x][y]

      #updating knowledge base
      map[x][y] = cell('M',9,map[x][y].idSafe, map[x][y].idMine, map[x][y].hidden, map[x][y].total)
      

    else:
      #check if anything can be determined from the clue and update knowledge base
      game[x][y] = grid[x][y]

      if grid[x][y] == 0:
        #generate neighbors and add all neighbors to the fringe
        fringe = generateNeighbors(x, y, grid, fringe, explored)
        

      #updating the knowledge base
      hidden, safe, mine = findNeighbors(game, map, x, y)
      map[x][y] = cell('S',grid[x][y],safe,mine,hidden, map[x][y].total)
    
    #update this cell's neighbors with information discovered from the clue (ie. mine or safe)
    map, unexplored, explored, fringe, game = updateNeighbors(game, map, x, y, unexplored, explored, fringe)
    map, unexplored, explored, fringe, game, bombs = scanBoard(map, unexplored, explored, fringe, game, bombs)   


    #adding this cell to explored
    unexplored.remove((x,y))
    explored.add((x,y))
   # print(game)

  #print(game)
  return explodedBombs

  

grid = genGrid(10,40)
print(grid)
basicAgent(grid)

[[1. 3. 9. 2. 0. 1. 2. 3. 3. 2.]
 [9. 5. 9. 4. 2. 2. 9. 9. 9. 9.]
 [4. 9. 9. 9. 2. 9. 6. 9. 6. 3.]
 [9. 9. 9. 3. 3. 3. 9. 9. 5. 9.]
 [9. 9. 5. 2. 3. 9. 6. 9. 6. 9.]
 [4. 9. 5. 9. 5. 9. 6. 9. 9. 3.]
 [2. 9. 9. 3. 9. 9. 9. 3. 4. 9.]
 [2. 4. 3. 4. 3. 4. 2. 1. 3. 9.]
 [9. 3. 9. 2. 9. 1. 0. 0. 2. 9.]
 [2. 9. 2. 2. 1. 1. 0. 0. 1. 1.]]


18

# Advanced Agent

In [ ]:
class advCell:
  def __init__(self, status, clue, idSafe, idMine, hidden, neighbors): 
    self.status = status #covered, safe, mine, etc
    self.clue = clue #0-8
    self.idSafe = idSafe #number of known safe cells
    self.idMine = idMine #number of known mines
    self.hidden = hidden #active list of hidden neighbors
    self.neighbors = neighbors  #original list of neighbors
    self.valid = [0,1] #valid possible values




In [ ]:
#this function finds all neighbors of a given coordinate
def findAllNeighbors(row,col,d):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1)]
  result = []
  for (dx,dy) in movements:
      xx = row + dx
      yy = col + dy
      # check boundary
      if xx < 0 or xx >= d:
          continue
      if yy < 0 or yy >= d:
          continue
      result.append((xx,yy))

  return result

In [ ]:
#this function flags neighbors of a given coordinats as mines or as safe
def advFlag(map,x,y,value,unexplored,explored,fringe,bombs,game, validDF):
  movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1)]
  for i in range(len(map[x][y].hidden)):
    neighbor = map[x][y].hidden[i]

    #if the neighbor needs to be flagged as a mine
    if (value == 0 and neighbor not in bombs):
      
      bombs.append(neighbor)
      mine = neighbor

      map[mine[0]][mine[1]].status = 'M'
      map[mine[0]][mine[1]].clue = 9
      game[mine[0]][mine[1]] = 10

      #check if this neighbor is in the current list of variables and remove if it is
      if (str(neighbor) in validDF.columns):
        validDF = validDF.drop(str(neighbor), axis = 1)
      

      for j in range(len(map[mine[0]][mine[1]].neighbors)):
        cell = map[mine[0]][mine[1]].neighbors[j]
        map[cell[0]][cell[1]].idMine += 1

        if cell == (x,y):
          continue

        map[cell[0]][cell[1]].hidden.remove(mine)
      
      unexplored.remove(mine)
      explored.append(mine)

    #if the neighbor needs to be flagged as safe
    elif (value == 1 and neighbor not in fringe and neighbor not in explored): 
      fringe.append(neighbor)
      
    


  return map, unexplored, explored, fringe, bombs, game, validDF




In [ ]:
#this function helps find a coordinate
def advFindCoordinate(map, unexplored):
  count = 0
  row = -1
  col = -1
  minimum = sys.maxsize

  for i in range(len(unexplored)):
    coordinate = unexplored[i]
    x = coordinate[0]
    y = coordinate[1]

    if len(map[x][y].hidden) == len(map[x][y].neighbors):
      row = x
      col = y
      break
    
    #sort unexplored coordinates by amount of neighboring mines that still need to be discovered
    else:
      total = 0;
      movements = [(1,0),(-1,0),(0,1),(0,-1),(1,1),(-1,-1),(-1,1),(1,-1)]

      for j in range(len(movements)):
        distX, distY = movements[j]
        xx = x + distX
        yy = y + distY 

        if (xx<0 or yy < 0 or xx >= len(map) or yy >= len(map)):
          continue

        #if the neighbor is safe, calculate how many surrounding mines still need to be found
        if map[xx][yy].status == 'S':
          total += map[xx][yy].clue - map[xx][yy].idMine
          
      #update row and col to the x and y with the neighbors that have the least amount of mines still hidden
      if minimum > total:
        minimum = total
        row = x
        col = y

  return row, col

In [ ]:
#this function goes through the current knowledge base and determines if anything can be determined
def advScanBoard(kb, map, game, d, fringe, unexplored, explored, bombs, validDF):
  for clause in kb:
    
    x = clause[0]
    y = clause[1]
    clue = clause[2]
    #check local clues to determine if anything is mine
    if map[x][y].clue - map[x][y].idMine == len(map[x][y].hidden):
      #flag remaining hidden neighbors as bombs
      map, unexplored, explored, fringe, bombs, game, validDF = advFlag(map, x, y, 0, unexplored, explored, fringe, bombs, game, validDF) #write a flag function to handle all the map edits
      map[x][y].hidden.clear()
      
  
    #check local clues to determine if anything is safe
    elif (len(map[x][y].neighbors) - map[x][y].clue) - map[x][y].idSafe == len(map[x][y].hidden):
      #flag remaining neighbors as safe
      map, unexplored, explored, fringe, bombs, game, validDF = advFlag(map, x, y, 1, unexplored, explored, fringe, bombs, game, validDF) #flag function to handle all map edits 
      break
      
    #check all remaining combinations against the current knowledge base clause and see if anything can be determined
    else:
      strHiddenNeigh = [str(i) for i in map[x][y].hidden]
      hiddenNeigh = pd.DataFrame(columns = strHiddenNeigh)

      #get the combinations of neighbors that are already in the valid combinations dataframe
      shared = np.intersect1d(validDF.columns,hiddenNeigh.columns)
      if (len(shared)!=0):
      
        shared = [str(i) for i in shared]
        xDF = validDF.drop_duplicates(shared)
        xDF = xDF[shared]

      else:
        xDF = pd.DataFrame(columns = [])
      
      #set difference for coords you have to make binary comb for
      notShared = (list(list(set(strHiddenNeigh)-set(shared)) + list(set(shared)-set(strHiddenNeigh))))  
 
      #create binary combinations for all new neighbors
      if len(notShared) != 0:
        notSharedDF = pd.DataFrame(columns = notShared)
        for i in range(2**len(notShared)): #creating binary comb
          binary = format(i, '#0%db' % (len(notShared)+2))
          binaryArr = []
          for j in range(2,len(binary)):
            binaryArr.append(int(binary[j]))
       
          notSharedDF.loc[len(notSharedDF.index)] = binaryArr #adding binary comb to row by column index

        #if there are shared neighbors, merge accordingly and keep combinations that satisfy the clue
        if (len(xDF.columns) != 0):
          xDF['key'] = 1
          notSharedDF['key'] = 1

          mergeDF = pd.merge(xDF, notSharedDF, on ='key').drop("key", 1)
          mergeDF['Total'] = mergeDF.sum(axis=1)
          mergeDF = mergeDF[mergeDF['Total'] == clue - map[x][y].idMine]
          mergeDF = mergeDF.drop(['Total'], axis = 1)

        #if there are no shared neighbors, merge accordingly and keep combinations that satisfy the clue
        else:
          mergeDF = notSharedDF.copy()
          '''
          mergeDF['Total'] = mergeDF.sum(axis=1)
          mergeDF = mergeDF[mergeDF['Total'] == clue - map[x][y].idMine]
          mergeDF = mergeDF.drop(['Total'], axis = 1)
          '''
      #if all neighbors are shared, merge accordingly and keep combinations that satisfy the clue
      else:
        mergeDF = xDF.copy()
        mergeDF['Total'] = mergeDF.sum(axis=1)
        mergeDF = mergeDF[mergeDF['Total'] == clue - map[x][y].idMine]
        mergeDF = mergeDF.drop(['Total'], axis = 1)

      #add new combinations to the valid combinations dataframe
      shared = list(shared)

      if (len(shared)!= 0):
        if (len(validDF.columns)==0):
          validDF = mergeDF.copy()

        else:
          validDF = validDF.merge(mergeDF, left_on=shared, right_on=shared)

      else:

        if (len(validDF.columns) == 0):
          validDF = mergeDF.copy()

        else:
          validDF['key'] = 1
          mergeDF['key'] = 1
          validDF = pd.merge(validDF, mergeDF, on ='key').drop("key", 1) 

      #go through validDF and determine if anything can be concluded about a coordinate
      count = validDF.nunique()
      for index, value in count.items():
        if (value == 1):
          cellType = validDF[index][0]

          #if the coordinate is safe, add to fringe
          if (cellType == 0 and eval(index) not in fringe and eval(index) not in explored):
            #print(f'Marking {index} as safe in dataframe')
            fringe.append(eval(index))
            
          #if the coordinate is a mine, flag it as a mine, remove it from its neighbors hidden, and remove it from validDF
          elif (cellType == 1):

            if (eval(index) not in bombs):
              intIndex = eval(index)
              bombs.append(intIndex)

              map[intIndex[0]][intIndex[1]].status = 'M'
              map[intIndex[0]][intIndex[1]].clue = 9
              game[intIndex[0]][intIndex[1]] = 10
              map[intIndex[0]][intIndex[1]].valid.remove(0)

              unexplored.remove(intIndex)
              explored.append(intIndex)

              for i in range(len(map[intIndex[0]][intIndex[1]].neighbors)):
                point = map[intIndex[0]][intIndex[1]].neighbors[i]
                map[point[0]][point[1]].idMine += 1

                if (intIndex in map[point[0]][point[1]].hidden):
                  map[point[0]][point[1]].hidden.remove(intIndex)
                  
              validDF = validDF.drop([index],axis=1)
        
  return map, unexplored, explored, fringe, bombs, game, kb, validDF

In [ ]:
#this is the advanced agent function
def advAgent(grid):

  validDF = pd.DataFrame(columns=[])
  d = len(grid)
  map = np.zeros((d,d), dtype='object')
  unexplored = []

  #create a map to keep track of the status, the clue, the number of identified mines, the number of identified safe, a list of hidden neighbors, and a list of all neighbors
  for row in range(d):
    for col in range(d):
      result = findAllNeighbors(row,col,d)
      map[row][col] = advCell('C',-1,0,0,result,result.copy())
      unexplored.append((row,col))


  explored = [] #will store all explored coordinates


  #build the game board
  game = np.zeros((len(grid),len(grid))) 
  game[:] = -1
  bombs = [] #will store all known/determined bomb locations 
  fringe = [] #will store all coordinates that can be safely explored 
  kb = [] #will store coordinate plus clue

  explodedBombs = 0
  #continue this loop until all coordinates have been explored and there are no coordinates in the fringe
  while (len(unexplored) != 0 or len(fringe) != 0):
    
    #if fringe is empty, find a random coordinate
    if (len(fringe)==0):
      #print("random coordinate")
      num = random.randint(0, len(unexplored)-1)
      coord = unexplored[num]
      x = coord[0]
      y = coord[1]

    #if fringe is not empty, pop from fringe
    else:
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

 

    game[x][y] = grid[x][y]
   
    #if the selected coordinate is a mine, update the map, validDF, and its neighbors
    if (grid[x][y] == 9):

     
      bombs.append((x,y))
      explodedBombs += 1
      map[x][y].status = 'M'
      map[x][y].clue = 9

   
      #remove selected coordinate from validDF since we know it's a mine
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)

      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idMine += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #if the selected coordinate is safe
    else:

      #if the coordinate has a clue of 0, add all neighbors to its fringe
      if (grid[x][y]==0):

        for i in range(len(map[x][y].hidden)):
          point = map[x][y].hidden[i]
          if (point not in fringe and point not in explored):  
            fringe.append(map[x][y].hidden[i])
           
      #if the coordinate has a clue of 1-8, add it to the knowledge base
      else:

        if len(map[x][y].hidden) != 0:
          kb.append((x,y,grid[x][y]))
          kb = sorted(kb, key = lambda kb: kb[2], reverse = True)
          result = [] 
          [result.append(x) for x in kb  if x not in result]
          kb = result.copy()
          #print(kb)

      #remove selected coordinate from validDF since we know its safe
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)  

      #update the status of the current coordinate and update the coordinate's neighbors
      map[x][y].status = 'S'
      map[x][y].clue = grid[x][y]
     

      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idSafe += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #add the selected coordinate to explored and remove it from unexplored
    unexplored.remove((x,y))
    explored.append((x,y))

    #if the fringe is empty, scan the board to see if anything can be determined with the clues in the current knowledge base
    if len(fringe) == 0:
      map, unexplored, explored, fringe, bombs, game, kb, validDF = advScanBoard(kb, map, game, d, fringe, unexplored, explored, bombs, validDF)
      i = 0

      #go through the knowledge base and remove any clauses with coordinates that have no more hidden neighbors (nothing more can be determined with this clause) 
      while i < len(kb):
        if len(map[kb[i][0]][kb[i][1]].hidden) == 0:
          kb.remove(kb[i])
          i = i-1
        i = i+1

  return explodedBombs
      
    
    
      
grid = genGrid(10,15)
print(grid)
advAgent(grid)

  

[[1, 9, 2], [3, 4, 9], [9, 3, 9]]


1

# Performance

In [ ]:
#play by play for 5x5 grid
grid = genGrid(5, 5)
print(grid)
print("Playing as Basic Agent")
basicAgent(grid)
print("Playing as Advanced Agent")
advAgent(grid)

[[1. 9. 9. 1. 0.]
 [1. 2. 2. 1. 0.]
 [0. 1. 1. 1. 0.]
 [0. 2. 9. 3. 1.]
 [0. 2. 9. 3. 9.]]
Playing as Basic Agent
Playing as Advanced Agent


1

In [ ]:
#comparing differences between advanced and basic agents
density = 3
basicResults = []
advResults = []
basicTotal = 0
advTotal = 0

print(f'testing {density} mines')
for j in range(10):
  print(f'on iteration {j}')
  grid = genGrid(10, density)
  basicTotal += basicAgent(grid)
  print(f'checking advanced')
  advTotal += advAgent(grid)
advAvg = advTotal/10
basicAvg = basicTotal/10
print(f'basic agent: {(density-basicAvg)/density}')
print(f'advanced agent: {(density-advAvg)/density}')

testing 3 mines
on iteration 0
checking advanced
on iteration 1
checking advanced
on iteration 2
checking advanced


KeyboardInterrupt: ignored

In [ ]:
#comparing differences between advanced and basic agents

density = [3,6,9,12,15,18,21,24,27,30]
basic = [0.93, 0.93, 0.96, 0.95, 0.9, 0.93, 0.90, 0.89, 0.85, 0.8  ]
adv = [0.97, 0.99, 0.99, 0.98, 0.98, 0.98, 0.95, 0.94, 0.92, 0.89]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.plot(density, basic, marker = ".", markersize=10)
ax.plot(density, adv, marker = ".", markersize=10)
ax.set_xlabel('Number of Mines')
ax.set_ylabel('Difference between basic and advance agent')
ax.set_title("Performance")
ax.legend(labels = ["Basic Agent", "Advanced Agent"])
fig.show()

# Global Information

In [ ]:
def globalBasicAgent(grid, x):
  unexplored = []
  for row in range(len(grid)): 
    for col in range(len(grid[0])):
      unexplored.append((row,col))

  explored = set([])
  explodedBombs = 0
  bombs = []

  fringe = []
  d = len(grid)
  game = np.zeros((d,d)) 
  game[:] = -1
  
  #make a map of cells to represent the knowledge base
  map = np.zeros((d,d), dtype='object')
  for row in range(d):
    for col in range(d):
      if ((row, col) == (0, 0)) or  ((row, col) == (0, d-1)) or ((row, col) == (d-1, 0)) or ((row, col) == (d-1, d-1)):
        map[row][col] = cell('C',0,0,0, 3, 3)
      elif (row == 0 or row == d-1 or col == 0 or col == d-1):
        map[row][col] = cell('C',0,0,0, 5, 5) 
      else:
        map[row][col] = cell('C',0,0,0, 8, 8)
  
  totalBombs = x
  while len(explored) != d*d:
    #all mines have been found, open all unexplored cells and then break out of loop
    if len(bombs) == totalBombs:
      for (xx, yy) in unexplored:
        game[xx][yy]=grid[xx][yy]
      break

    #all the coordinates left in unexplored are mines, update the game board and break out of loop
    if totalBombs - len(bombs) == len(unexplored):
      for (xx, yy) in unexplored:
        game[xx][yy]=10
      break

    #there are no known safe cells
    if (len(fringe)==0):
      num = random.randint(0, len(unexplored)-1)
      coord = unexplored[num]
      x = coord[0]
      y = coord[1]

    #there are still safe cells left to explore
    else:
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

    if (x,y) in explored:
      continue

    if map[x][y].status == 'M':
      continue
    
    if grid[x][y] == 9:
      explodedBombs += 1
      game[x][y] = grid[x][y]
      bombs.append((x,y))

      #updating knowledge base
      map[x][y] = cell('M',9,map[x][y].idSafe, map[x][y].idMine, map[x][y].hidden, map[x][y].total)
      

    else:
      #check if anything can be determined from the clue and update knowledge base
      game[x][y] = grid[x][y]

      if grid[x][y] == 0:
        #generate neighbors and add all neighbors to the fringe
        fringe = generateNeighbors(x, y, grid, fringe, explored)
        

      #updating the knowledge base
      hidden, safe, mine = findNeighbors(game, map, x, y)
      map[x][y] = cell('S',grid[x][y],safe,mine,hidden, map[x][y].total)
    
    #update this cell's neighbors with information discovered from the clue (ie. mine or safe)
    map, unexplored, explored, fringe, game = updateNeighbors(game, map, x, y, unexplored, explored, fringe)
    map, unexplored, explored, fringe, game, bombs = scanBoard(map, unexplored, explored, fringe, game, bombs)   


    #adding this cell to explored
    unexplored.remove((x,y))
    explored.add((x,y))

  return explodedBombs


In [ ]:
def globalAdvAgent(grid, x):
  validDF = pd.DataFrame(columns=[])
  d = len(grid)
  map = np.zeros((d,d), dtype='object')
  unexplored = []

  #create a map to keep track of the status, the clue, the number of identified mines, the number of identified safe, a list of hidden neighbors, and a list of all neighbors
  for row in range(d):
    for col in range(d):
      result = findAllNeighbors(row,col,d)
      map[row][col] = advCell('C',-1,0,0,result,result.copy())
      unexplored.append((row,col))


  explored = []


  #build the game board
  game = np.zeros((len(grid),len(grid))) 
  game[:] = -1
  bombs = []
  fringe = []
  kb = [] #will store coordinate plus clue
  totalBombs = x
  explodedBombs = 0

  #continue this loop until all coordinates have been explored and there are no coordinates in the fringe
  while (len(unexplored) != 0 or len(fringe) != 0): 
   #all mines have been found, open all unexplored cells and then break out of loop
    if len(bombs) == totalBombs:
      for (xx, yy) in unexplored:
        game[xx][yy]=grid[xx][yy]
      break

    #all the coordinates left in unexplored are mines, update the game board and break out of loop
    if totalBombs - len(bombs) == len(unexplored):
      for (xx, yy) in unexplored:
        game[xx][yy]=10
      break

    #if fringe is empty, find a random coordinate
    if (len(fringe)==0):
      num = random.randint(0, len(unexplored)-1)
      coord = unexplored[num]
      x = coord[0]
      y = coord[1]


    #if fringe is not empty, pop from fringe
    else:
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

    game[x][y] = grid[x][y]
   
    #if the selected coordinate is a mine, update the map, validDF, and its neighbors
    if (grid[x][y] == 9):
      bombs.append((x,y))
      explodedBombs += 1
      map[x][y].status = 'M'
      map[x][y].clue = 9
      #remove selected coordinate from validDF since we know it's a mine
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)
      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idMine += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #if the selected coordinate is safe
    else:
      #if the coordinate has a clue of 0, add all neighbors to its fringe
      if (grid[x][y]==0):
        for i in range(len(map[x][y].hidden)):
          point = map[x][y].hidden[i]
          if (point not in fringe and point not in explored):  
            fringe.append(map[x][y].hidden[i])
           
      #if the coordinate has a clue of 1-8, add it to the knowledge base
      else:
        if len(map[x][y].hidden) != 0:
          kb.append((x,y,grid[x][y]))
          kb = sorted(kb, key = lambda kb: kb[2], reverse = True)
          result = [] 
          [result.append(x) for x in kb  if x not in result]
          kb = result.copy()
          #print(kb)

      #remove selected coordinate from validDF since we know its safe
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)  

      #update the status of the current coordinate and update the coordinate's neighbors
      map[x][y].status = 'S'
      map[x][y].clue = grid[x][y]
      #map[x][y].valid.remove(1)
      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idSafe += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #add the selected coordinate to explored and remove it from unexplored
    unexplored.remove((x,y))
    explored.append((x,y))

    #if the fringe is empty, scan the board to see if anything can be determined with the clues in the current knowledge base
    if len(fringe) == 0:
      map, unexplored, explored, fringe, bombs, game, kb, validDF = advScanBoard(kb, map, game, d, fringe, unexplored, explored, bombs, validDF)
      i = 0

      #go through the knowledge base and remove any clauses with coordinates that have no more hidden neighbors (nothing more can be determined with this clause) 
      while i < len(kb):
        if len(map[kb[i][0]][kb[i][1]].hidden) == 0:
          kb.remove(kb[i])
          i = i-1
        i = i+1

  
  return explodedBombs


In [ ]:
#comparing differences between advanced and basic agents with global information

density = 3
basicResults = []
advResults = []
basicTotal = 0
advTotal = 0

print(f'testing {density} mines')
for j in range(10):
  print(f'on iteration {j}')
  grid = genGrid(10, density)
  basicTotal += globalBasicAgent(grid, density)
  print(f'checking advanced')
  advTotal += globalAdvAgent(grid, density)
advAvg = advTotal/10
basicAvg = basicTotal/10
print(f'basic agent: {(density-basicAvg)/density}')
print(f'advanced agent: {(density-advAvg)/density}')


In [ ]:
#comparing differences between advanced and basic agents with global information

density = [3,6,9,12,15,18,21,24,27,30]
basic = [1, 0.98, 0.97, 0.94, 0.83, 0.89, 0.86, 0.83, 0.81, 0.79]
adv = [1, 1, 0.99, 0.95, 0.97, 0.95, 0.94, 0.92, 0.92, 0.9]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.plot(density, basic, marker = ".", markersize=10)
ax.plot(density, adv, marker = ".", markersize=10)
ax.set_xlabel('Number of Mines')
ax.set_ylabel('Difference between basic and advance agent')
ax.set_title("Global Information Performance")
ax.legend(labels = ["Basic Agent", "Advanced Agent"])
fig.show()






# Better Decisions

In [ ]:
def notRandomBasicAgent(grid):
  unexplored = []
  for row in range(len(grid)): 
    for col in range(len(grid[0])):
      unexplored.append((row,col))

  explored = set([])
  explodedBombs = 0
  bombs = []

  fringe = []
  d = len(grid)
  game = np.zeros((d,d)) 
  game[:] = -1

  #make a map of cells to represent the knowledge base
  map = np.zeros((d,d), dtype='object')
  for row in range(d):
    for col in range(d):
      if ((row, col) == (0, 0)) or  ((row, col) == (0, d-1)) or ((row, col) == (d-1, 0)) or ((row, col) == (d-1, d-1)):
        map[row][col] = cell('C',0,0,0, 3, 3)
      elif (row == 0 or row == d-1 or col == 0 or col == d-1):
        map[row][col] = cell('C',0,0,0, 5, 5) 
      else:
        map[row][col] = cell('C',0,0,0, 8, 8)
  
  
  while len(explored) != d*d:

    #there are no known safe cells
    if (len(fringe)==0):
      x, y = findCoordinate(map, unexplored)

    #there are still safe cells left to explore
    else:
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

    if (x,y) in explored:
      continue

    if map[x][y].status == 'M':
      continue
    
    if grid[x][y] == 9:
      explodedBombs += 1
      game[x][y] = grid[x][y]

      #updating knowledge base
      map[x][y] = cell('M',9,map[x][y].idSafe, map[x][y].idMine, map[x][y].hidden, map[x][y].total)
      

    else:
      #check if anything can be determined from the clue and update knowledge base
      game[x][y] = grid[x][y]

      if grid[x][y] == 0:
        #generate neighbors and add all neighbors to the fringe
        fringe = generateNeighbors(x, y, grid, fringe, explored)
        

      #updating the knowledge base
      hidden, safe, mine = findNeighbors(game, map, x, y)
      map[x][y] = cell('S',grid[x][y],safe,mine,hidden, map[x][y].total)
    
    #update this cell's neighbors with information discovered from the clue (ie. mine or safe)
    map, unexplored, explored, fringe, game = updateNeighbors(game, map, x, y, unexplored, explored, fringe)
    map, unexplored, explored, fringe, game, bombs = scanBoard(map, unexplored, explored, fringe, game, bombs)   


    #adding this cell to explored
    unexplored.remove((x,y))
    explored.add((x,y))
  

  return explodedBombs

  



In [ ]:
def notRandomAdvAgent(grid):
  validDF = pd.DataFrame(columns=[])
  d = len(grid)
  map = np.zeros((d,d), dtype='object')
  unexplored = []

  #create a map to keep track of the status, the clue, the number of identified mines, the number of identified safe, a list of hidden neighbors, and a list of all neighbors
  for row in range(d):
    for col in range(d):
      result = findAllNeighbors(row,col,d)
      map[row][col] = advCell('C',-1,0,0,result,result.copy())
      unexplored.append((row,col))


  explored = []


  #build the game board
  game = np.zeros((len(grid),len(grid))) 
  game[:] = -1
  bombs = []
  fringe = []
  kb = [] #will store coordinate plus clue

  explodedBombs = 0
  #continue this loop until all coordinates have been explored and there are no coordinates in the fringe
  while (len(unexplored) != 0 or len(fringe) != 0):
    
    #if fringe is empty, find a random coordinate
    if (len(fringe)==0):
      #print("random coordinate")
      x, y = advFindCoordinate(map, unexplored)

    #if fringe is not empty, pop from fringe
    else:
      coord = fringe.pop()
      x = coord[0]
      y = coord[1]

    #print((x,y))
    #print(fringe)
    #print(bombs)

    game[x][y] = grid[x][y]
   
    #if the selected coordinate is a mine, update the map, validDF, and its neighbors
    if (grid[x][y] == 9):
      #print("Bomb exploded")
      bombs.append((x,y))
      explodedBombs += 1
      map[x][y].status = 'M'
      map[x][y].clue = 9
      #map[x][y].valid.remove(0)
      #remove selected coordinate from validDF since we know it's a mine
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)
      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idMine += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #if the selected coordinate is safe
    else:
      #if the coordinate has a clue of 0, add all neighbors to its fringe
      if (grid[x][y]==0):
        for i in range(len(map[x][y].hidden)):
          point = map[x][y].hidden[i]
          if (point not in fringe and point not in explored):  
            fringe.append(map[x][y].hidden[i])
           
      #if the coordinate has a clue of 1-8, add it to the knowledge base
      else:
        if len(map[x][y].hidden) != 0:
          kb.append((x,y,grid[x][y]))
          kb = sorted(kb, key = lambda kb: kb[2], reverse = True)
          result = [] 
          [result.append(x) for x in kb  if x not in result]
          kb = result.copy()
          #print(kb)

      #remove selected coordinate from validDF since we know its safe
      if (str((x,y)) in validDF.columns):
        validDF = validDF.drop([str((x,y))], axis = 1)  

      #update the status of the current coordinate and update the coordinate's neighbors
      map[x][y].status = 'S'
      map[x][y].clue = grid[x][y]
      #map[x][y].valid.remove(1)
      for i in range(len(map[x][y].neighbors)):
        point = map[x][y].neighbors[i]
        map[point[0]][point[1]].idSafe += 1
        map[point[0]][point[1]].hidden.remove((x,y))
    
    #add the selected coordinate to explored and remove it from unexplored
    unexplored.remove((x,y))
    explored.append((x,y))

    #if the fringe is empty, scan the board to see if anything can be determined with the clues in the current knowledge base
    if len(fringe) == 0:
      map, unexplored, explored, fringe, bombs, game, kb, validDF = advScanBoard(kb, map, game, d, fringe, unexplored, explored, bombs, validDF)
      i = 0

      #go through the knowledge base and remove any clauses with coordinates that have no more hidden neighbors (nothing more can be determined with this clause) 
      while i < len(kb):
        if len(map[kb[i][0]][kb[i][1]].hidden) == 0:
          kb.remove(kb[i])
          i = i-1
        i = i+1

    
    
    #print(map[x][y].hidden)
    #print(map[x][y].neighbors)
    #print(game)
  return explodedBombs

In [ ]:
#comparing differences between advanced and basic agents with better decisions


density = 3
basicResults = []
advResults = []
basicTotal = 0
advTotal = 0

print(f'testing {density} mines')
for j in range(10):
  print(f'on iteration {j}')
  grid = genGrid(10, density)
  basicTotal += notRandomBasicAgent(grid)
  print(f'checking advanced')
  advTotal += notRandomAdvAgent(grid)
advAvg = advTotal/10
basicAvg = basicTotal/10
print(f'basic agent: {(density-basicAvg)/density}')
print(f'advanced agent: {(density-advAvg)/density}')




In [ ]:
#comparing differences between advanced and basic agents with better decisions

density = [3,6,9,12,15,18,21,24,27,30]
basic = [0.97, 0.98, 0.97, 0.94, 0.89, 0.90, 0.86, 0.79, 0.81, 0.61 ]
adv = [0.97, 0.98, 0.99, 0.95, 0.93, 0.96, 0.96, 0.96, 0.96, 0.88 ]

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.plot(density, basic, marker = ".", markersize=10)
ax.plot(density, adv, marker = ".", markersize=10)
ax.set_xlabel('Number of Mines')
ax.set_ylabel('Difference between basic and advance agent')
ax.set_title("Better Decisions Performance")
ax.legend(labels = ["Basic Agent", "Advanced Agent"])
fig.show()

# Citations

“Merge, Join, Concatenate and Compare.” Merge, Join, Concatenate and Compare - Pandas 1.2.3 Documentation, pandas.pydata.org/docs/user_guide/merging.html. \\

“Pandas.DataFrame.” Pandas.DataFrame - Pandas 1.2.3 Documentation, pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html. \\

Real Python. “Object-Oriented Programming (OOP) in Python 3.” Real Python, Real Python, 16 Jan. 2021, realpython.com/python3-object-oriented-programming/. 